In [ ]:
import xarray as xr
from emit_tools import emit_xarray
import os
from fsspec.implementations.http import HTTPFileSystem
import holoviews as hv
import hvplot.xarray  # noqa: F401

from utils import band_index, gamma_adjust, get_rgb_dataset, build_interactive_map

# Starter Notebook

This notebook demonstrates some basic functionality that is useful in working with the EMIT data
from NASA.

## Loading data

To load data, there's a function provided by NASA that will restructure the NetCDF file,
which we can stream from their cloud data store.

To discover scenes, you can browse NASA's STAC catalog in the Radiant Earth [STAC Browser](https://radiantearth.github.io/stac-browser/#/external//cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/collections/EMITL2ARFL.v001).


In [ ]:
# Generate a user token and set it as an environment variable
# Tokens can be generated at https://urs.earthdata.nasa.gov/users/<username>/user_tokens
token = os.environ.get('EARTHDATA_TOKEN')
if token is None:
    raise ValueError('Please set the EARTHDATA_TOKEN environment variable')

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))
ds

## Render data as RGB and query point for spectra plot

This section will select three bands to visualise on the red, green and blue
channels of an image and will stretch the data to make it easier to view.

This plot is very simple and is a useful visual aid, but it's no longer data, it's a visualisation.

In [ ]:
# Select wavelengths to be displayed
r = 2100
g = 950
b = 490

# select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.4

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], brightness)

# Plot the RGB data
ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

## Run normalised band index

Here we're going to visualise some data and convert it into information in the form
of a true/false value for whether or not a pixel is water.

The logic here is to pick a band where the reflectance of a pixel over water is
high and where it's low, to then isolate those bands and finally do an NDWI calculation.

From [this reference](https://bmcecol.biomedcentral.com/articles/10.1186/s12898-019-0233-0/tables/2),
it's suggested to use something like `860` for the first value and
`1240`, `1640` or `2130` for the high value (short/long wavelengths). By inspecting bands,
I've picked `650` for the first band, as it has high values over water and `1275` as it
has low values over water. This should lead to water having values approaching 1, and
non-water with values approaching -1.

In [ ]:
# High meaning bright, or high reflectance
high = 450
low = 1275

ds["water_high"] = ds.reflectance.sel(bands=high, method="nearest")
ds["water_low"] = ds.reflectance.sel(bands=low, method="nearest")
ds["water_diff"] = ds.water_high - ds.water_low

ds["ndwi"] = ds.water_diff / (ds.water_high + ds.water_low)

In [ ]:
plot_options = {
    "aspect": "equal",
    "frame_width": 300,
    "xaxis": "bare",
    "yaxis": "bare",
}

image_high = ds.water_high.hvplot(**plot_options, title=f"high: {high}", clim=(0, 0.4), cmap="viridis")
image_low = ds.water_low.hvplot(**plot_options, title=f"low: {low}", clim=(0, 0.4), cmap="viridis")
image_diff = ds.water_diff.hvplot(**plot_options, title="difference", clim=(-1, 1), cmap="RdBu")
image_ndwi = ds.ndwi.hvplot(**plot_options, title="NDWI", clim=(-1, 1), cmap="bwy_r")


(image_high + image_low + image_diff + image_ndwi).cols(2)

In [ ]:
# Plot a histogram of the NDWI values, so we can pick a threshold
ds.ndwi.hvplot.hist()

In [ ]:
ds["threshold_water"] = ds.ndwi > 0.0
ds.threshold_water.hvplot()

# Save and load xarray

Saving work to disk as a cache or as an input to a separate process is simple.

In [ ]:
# Store xarray dataset on disk. The Canberra example was 5 GB when saved.
ds.to_netcdf("saved_on_disk.nc")

In [ ]:
# Load the xarray from disk again
ds_new = xr.open_dataset("saved_on_disk.nc")

## Exporting raster data

Finally, we'll export data in a standard geotiff format, which can then be used in
a GIS application or in web mapping.

In [ ]:
# Convert threshold_water band to uint8 and save to disk
ds.threshold_water.astype("uint8").rio.to_raster(f"water.tif")

In [ ]:
# Export a single band as a GeoTIFF
ds.isel(bands=band_index(ds, r)).reflectance.rio.to_raster(f"band_{r}.tif")
ds.isel(bands=band_index(ds, g)).reflectance.rio.to_raster(f"band_{g}.tif")
ds.isel(bands=band_index(ds, b)).reflectance.rio.to_raster(f"band_{b}.tif")

In [ ]:
# Combine three images into a single GeoTIFF with three bands
import osgeo_utils.gdal_merge

output_file_path = f"bands_{r}_{g}_{b}.tif"
input_files_path = f"band_{r}.tif band_{g}.tif band_{b}.tif".split()
parameters = ['', '-o', output_file_path] + input_files_path + ['-separate', '-co', 'COMPRESS=LZW']
osgeo_utils.gdal_merge.main(parameters)